In [3]:
# Welcome to Strangeworks Notebooks!
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
print("Imports Successful")

Imports Successful


In [4]:
# Method used to encode the bits with basis
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        elif bases[i] == 1: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        else: # Prepare qbit in Y-basis
            if bits[i] == 0:
                qc.y(0)
            else:
                qc.x(0)
                qc.y(0)
        qc.barrier()
        message.append(qc)
    return message

In [5]:
np.random.seed(seed=0)
n = 100
## Step 1
#Alice generates bits
alice_bits = randint(2, size=n)

## Step 2
# Create an array to tell us which qubits
# are encoded in which bases
alice_bases = randint(3, size=n)
print(alice_bases)

[1 2 0 0 2 0 0 0 0 0 0 2 0 2 1 1 1 0 1 1 1 0 1 2 0 1 2 0 2 0 1 2 2 1 0 1 1
 0 2 2 2 2 1 2 2 2 2 2 0 1 2 2 1 2 1 0 2 2 0 2 0 0 2 0 2 2 2 0 0 0 1 2 0 1
 2 2 2 1 0 0 0 0 2 2 0 0 0 1 2 0 0 1 0 2 1 1 1 0 0 0]


In [6]:
# Display a sample from the random bits and bases chosen
message = encode_message(alice_bits, alice_bases)
print('bit = %i' % alice_bits[1])
print('basis = %i' % alice_bases[1])
message[1].draw()

bit = 1
basis = 2


┌───┐┌───┐ ░ 
q_0: ┤ X ├┤ Y ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════

In [7]:
# Display a sample from the random bits and bases chosen
print('bit = %i' % alice_bits[4])
print('basis = %i' % alice_bases[4])
message[4].draw()

bit = 1
basis = 2


┌───┐┌───┐ ░ 
q_0: ┤ X ├┤ Y ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════

In [8]:
# Method used to measure the encrypted message
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        if bases[q] == 2: # measuring in Y-basis
            message[q].y(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [9]:
## Interception!!
# Eve intercepts the message and measures it
eve_bases = randint(2, size=n)
intercepted_message = measure_message(message, eve_bases)
print(intercepted_message)

[0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]


In [10]:
# The message after Eve measures it
message[0].draw()

┌───┐ ░ ┌───┐┌─┐
q_0: ┤ H ├─░─┤ H ├┤M├
     └───┘ ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

In [11]:
## Step 3
# Bob decides which basis to measure in:
bob_bases = randint(2, size=n)
print(bob_bases)

[1 1 1 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0
 1 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1
 0 1 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0]


In [12]:
# Display what Bob gets after measuring the encrypted message
bob_results = measure_message(message, bob_bases)
message[0].draw()

┌───┐ ░ ┌───┐┌─┐┌───┐┌─┐
q_0: ┤ H ├─░─┤ H ├┤M├┤ H ├┤M├
     └───┘ ░ └───┘└╥┘└───┘└╥┘
c: 1/══════════════╩═══════╩═
                   0       0

In [13]:
# Display another sample after Bob measures the encrypted message
message[6].draw()

┌───┐ ░ ┌───┐┌─┐┌─┐
q_0: ┤ X ├─░─┤ H ├┤M├┤M├
     └───┘ ░ └───┘└╥┘└╥┘
c: 1/══════════════╩══╩═
                   0  0

In [14]:
# Bobs full results
print(bob_results)

[1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1]


In [15]:
# Method used to remove bits measured with the incorrect basis
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [16]:
## Step 4 Alice removes bits measured with a basis that does not match Bobs
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
print(alice_key)

[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0]


In [17]:
# Bob removes bits measured with a basis that does not match Alice
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
print(bob_key)

[1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]


In [18]:
# Method used to compare random bits from Alice and Bobs secret key
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [19]:
## Step 5
# Display the random bits selected from Bob and Alice
sample_size = 15
bit_selection = randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))
# As you can see there is a large difference between the two samples

  bob_sample = [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]
alice_sample = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [20]:
# The secret keys do not match
bob_sample == alice_sample

False

In [21]:
# Eve's interference was detected
if bob_sample != alice_sample:
    print("Eve's interference was detected.")
else:
    print("Eve went undetected!")
    print(bob_key)
    print(alice_key)
    print("key length = %i" % len(alice_key))

Eve's interference was detected.
